# AI Engine入門チュートリアル

本チュートリアルでは簡単なデザインを使ってAI Engine開発の流れを体験します。ACRiブログの記事を参照しながら進めてください。

## 0. 準備
ソースコードやツールの出力を格納するディレクトリを作成します。

In [ ]:
!mkdir -p src data build

実行するターゲットをハードウェアエミュレーション（`hw_emu`）または実機（`hw`）から選択してください。

In [ ]:
import ipywidgets as widgets
w = widgets.RadioButtons(description='Target:', options=['hw_emu', 'hw'])
display(w)

選択した結果を後で参照できるようファイルに記録しておきます。

In [ ]:
import textwrap
with open('src/target.sh', 'w') as f:
    f.write(textwrap.dedent('''
    PLATFORM=xilinx_vck5000_gen4x8_qdma_2_202220_1
    TARGET={target}
    ''').format(target=w.value))

## 1. 作成するアプリケーション
このチュートリアルでは、int型の4要素のベクトルをふたつ足し合わせて結果を返すアプリを作成します。C++で次のように書けます。

In [ ]:
%%writefile src/app_org.cpp
#include <iostream>
#include <vector>

int main(int argc, char** argv)
{
    // 入力と出力のベクトル
    std::vector<int> in0(4), in1(4), out(4);
    
    // 適当な値でベクトルを初期化
    for (int i = 0; i < 4; i++) {
        in0[i] = i + 2;
        in1[i] = i * i;
    }
    
    // ベクトル加算
    for (int i = 0; i < 4; i++) {
        out[i] = in0[i] + in1[i];
    }
    
    // 結果を出力
    for (int i = 0; i < 4; i++) {
        std::cout << out[i] << std::endl;
    }
}

コンパイルして実行してみましょう。

In [ ]:
!g++ -o build/app_org src/app_org.cpp && ./build/app_org

このように出力されていればOKです。
```
2
4
8
14
```

## 2. 作成するデザイン
このチュートリアルで作成するデザインの全体像は次のようになります。

<div align="center"><img src="imgs/system.png" width="80%"/></div>

各部品とその役割は次の通りです。

- ホストCPUで実行するプログラム（app）
    - ホストメモリ、デバイスメモリにバッファを確保
    - 入力データを準備
    - ホストメモリとデバイスメモリ間のデータ転送
    - PLカーネル起動
- PLカーネル（mm2s、s2mm）
    - mm2s : メモリからデータを読み出し、ストリームとしてAIEへデータを転送
    - s2mm : AIEからストリームを受け取り、メモリへデータを書き出し
- AIEカーネル（vadd）、AIEグラフ（mygraph）
    - ストリームからベクトルを受け取り、加算、結果をストリームに送信

## 3. AIEカーネルの作成
AIEカーネルをC++で記述します。計算対象のふたつのベクトルを入力ストリームから読み込み、足し合わせて、出力ストリームに書き出します。

In [ ]:
%%writefile src/vadd.cpp
// 必要なヘッダーをインクルード
#include <aie_api/aie.hpp>
#include <aie_api/aie_adf.hpp>
#include <aie_api/utils.hpp>

// AIEカーネルの定義
void vadd(
    // 入力ストリーム
    input_stream<int32>* in0,
    input_stream<int32>* in1,
    
    // 出力ストリーム
    output_stream<int32>* out
) {
    // 入力ストリームからのデータをint32の4要素ベクトルとしてベクトルレジスタへ格納します
    aie::vector<int32, 4> a = readincr_v<4>(in0);
    aie::vector<int32, 4> b = readincr_v<4>(in1);
    
    // ベクトルレジスタに読み込んだふたつのベクトルを足し合わせます
    // 結果はベクトルレジスタに格納されます
    aie::vector<int32, 4> c = aie::add(a, b);
    
    // ベクトルレジスタ上の計算結果を出力ストリームに書き込みます
    writeincr(out, c);
}

AIEカーネルのヘッダーファイルを作成します。カーネルの関数宣言がAIEグラフの定義で参照されます。

In [ ]:
%%writefile src/vadd.hpp
#pragma once
#include <adf.h>

void vadd(
    input_stream<int32>* in0,
    input_stream<int32>* in1,
    output_stream<int32>* out
);

## 4. AIEグラフの作成
AIEグラフをC++で記述します。AIEグラフでは、AIEカーネルをインスタンスし、AIEカーネル間の接続と、AIEグラフの外との接続を定義します。作成するグラフは次の図のようになります。

<div align="center"><img src="imgs/graph.png" width="50%"/></div>

In [ ]:
%%writefile src/graph.hpp
#pragma once
// 必要なヘッダーをインクルード
#include <adf.h>

#include "vadd.hpp"

// adf::graphを継承したクラスを作成し、この中でグラフを定義します
class mygraph : public adf::graph
{
private:
    // AIEカーネルインスタンス
    adf::kernel vadd_kernel;

public:
    // PLとの入出力を定義
    adf::input_plio in0, in1;
    adf::output_plio out;

    // コンストラクタ
    mygraph()
    {
        // vaddカーネルを作成
        vadd_kernel = adf::kernel::create(vadd);
        // vaddカーネルのソースコードを指定
        adf::source(vadd_kernel) = "vadd.cpp";
        
        // PLとの入出力を具体的に定義します
        // 第一引数はポートの名前を設定します
        // 第二引数はポートのビット幅を設定します
        // 第三引数はシミュレーションで使用する入力または出力のファイル名を指定します
        in0 = adf::input_plio::create("in0", adf::plio_32_bits, "input0.txt");
        in1 = adf::input_plio::create("in1", adf::plio_32_bits, "input1.txt");
        out = adf::output_plio::create("out", adf::plio_32_bits, "output.txt");
        
        // カーネルとPLIOをストリームで接続します
        adf::connect<adf::stream>(in0.out[0], vadd_kernel.in[0]);
        adf::connect<adf::stream>(in1.out[0], vadd_kernel.in[1]);
        adf::connect<adf::stream>(vadd_kernel.out[0], out.in[0]);
        
        // カーネルのランタイム比を設定します
        adf::runtime<adf::ratio>(vadd_kernel) = 1.0;
    };
};

AIEグラフのシミュレーションを行うテストベンチを作成します。

In [ ]:
%%writefile src/graph.cpp
#include "graph.hpp"

mygraph graph;

int main(int argc, char** argv)
{
    graph.init();
    graph.run(1);
    graph.end();
}

## 5. AIEカーネル/グラフのコンパイルとシミュレーション

AIEグラフをハードウェアをターゲットとしてコンパイルします。

（Ubuntuでは/bin/shがdashになっているためエラーが出ます。ACRiルーム以外の環境で実行する場合は`sudo dpkg-reconfigure dash`でbashに変更してから実行してください。ACRiルームでは変更済みです）

In [ ]:
%%writefile src/aie_compile.sh
cd build

aiecompiler \
    --target=hw \
    --include=../src \
    ../src/graph.cpp

In [ ]:
!bash src/aie_compile.sh

コンパイラの出力としてlibadf.aが生成されているか確認します。

In [ ]:
!ls -l build/libadf.a

シミュレーション用の入力データを用意します。

In [ ]:
%%writefile data/input0.txt
2
3
4
5

In [ ]:
%%writefile data/input1.txt
0
1
4
9

シミュレーションを実行します。コンパイル時に生成されたWorkディレクトリの場所と、入力データを用意したディレクトリを引数で渡します。`--profile`オプションを指定することで、シミュレーション実行時にプロファイル情報を取得することができます。

In [ ]:
%%writefile src/aie_sim.sh
cd build

aiesimulator --pkg-dir=Work --input-dir=../data --profile

In [ ]:
!bash src/aie_sim.sh

シミュレーションにより出力されるファイルには出力時刻と値が記録されています。ファイルの中身を確認しましょう。

In [ ]:
!cat build/aiesimulator_output/output.txt

元のC++コードと同じ値を出力していれば期待通り動作していると言えます。

Vitis Analyzerを使ってコンパイルやプロファイルの結果を見てみましょう。

In [ ]:
%%sh
cd build
vitis_analyzer aiesimulator_output/default.aierun_summary &> /dev/null

## 6. PLカーネルの作成

メモリとAIEとの間でデータ移動を行うカーネルをC++で作成します。

mm2sは、第三引数で指定された数だけポインタを介してメモリからデータを読み出し、AXIストリームに書き込みます。

In [ ]:
%%writefile src/mm2s.hpp
#pragma once
#include <ap_int.h>
#include <ap_axi_sdata.h>
#include <hls_stream.h>

extern "C" {

void mm2s(
    ap_int<32>* mem,
    hls::stream<ap_axis<32, 0, 0, 0>>& str,
    int size
);

}

In [ ]:
%%writefile src/mm2s.cpp
#include "mm2s.hpp"

void mm2s(
    ap_int<32>* mem,
    hls::stream<ap_axis<32, 0, 0, 0>>& str,
    int size
) {
    for (int i = 0; i < size; i++)
    {
        ap_axis<32, 0, 0, 0> x;
        x.data = mem[i];
        x.keep = -1;
        str.write(x);
    }
}

s2mmはmm2sの逆を行います。

In [ ]:
%%writefile src/s2mm.hpp
#pragma once
#include <ap_int.h>
#include <ap_axi_sdata.h>
#include <hls_stream.h>

extern "C" {

void s2mm(
    ap_int<32>* mem,
    hls::stream<ap_axis<32, 0, 0, 0>>& str,
    int size
);

}

In [ ]:
%%writefile src/s2mm.cpp
#include "s2mm.hpp"

void s2mm(
    ap_int<32>* mem,
    hls::stream<ap_axis<32, 0, 0, 0>>& str,
    int size
) {
    for (int i = 0; i < size; i++)
    {
        auto x = str.read();
        mem[i] = x.data;
    }
}

ここではこれらのPLカーネルの検証は省略します。

これらのコードをVitisを使ってXilinx Objectにコンパイルします。このときC++コードが高位合成によりハードウェアに変換されます。

In [ ]:
%%writefile src/build_xo.sh
cd build

source ../src/target.sh

for kernel in mm2s s2mm ; do
    v++ \
        --compile -g \
        --target $TARGET \
        --platform $PLATFORM \
        --kernel $kernel \
        -I../src \
        -o $kernel.xo \
        ../src/$kernel.cpp
done

In [ ]:
!bash src/build_xo.sh

mm2s、s2mmそれぞれのxoファイルができていることを確認します。

In [ ]:
!ls build/*.xo

## 7. ハードウェアリンク

AIEグラフとPLカーネルができたら、デバイス側の部品が揃いますので、ひとつのシステムとしてリンクします。AIEグラフの記述と同様に、PL領域にPLカーネルをインスタンスし、各カーネルのストリームポート間の接続を指示します。今回のデザインでは次の図の構成となります。

<div align="center"><img src="imgs/vitis-link.png" width="60%"/></div>

Vitisではこれを設定ファイルとして記述します。設定ファイルの`connectivity`セクションに、インスタンスするPLカーネル名とその数を`nk`オプションで、カーネル間のストリーム接続を`sc`オプションで記述します。

mm2sは入力ベクトルのふたつ分インスタンスしていることに注目してください。AIEグラフの入出力は`ai_engine_0`にポートの名前を付けて接続します。

In [ ]:
%%writefile src/system.cfg
[connectivity]
nk=mm2s:2
nk=s2mm:1

sc=mm2s_1.str:ai_engine_0.in0
sc=mm2s_2.str:ai_engine_0.in1
sc=ai_engine_0.out:s2mm_1.str

[profile]
data=all:all:all

[vivado]
prop=fileset.sim_1.xsim.elaborate.xelab.more_options={-override_timeprecision -timescale=1ns/1ps}

Vitisのコマンドを使い、AIEグラフとPLカーネルをプラットフォームとリンクします。ターゲットがハードウェアエミュレーションの場合は15分ほど、実機の場合は50分ほどかかります。

pre-builtディレクトリ以下にビルド済みのxclbinファイルを用意しています。時間がないときは[9. ホストプログラムの作成](#9.-ホストプログラムの作成)までジャンプしてください。

In [ ]:
%%writefile src/link.sh
cd build

source ../src/target.sh

time v++ \
    --link -g \
    --target $TARGET \
    --platform $PLATFORM \
    --config ../src/system.cfg \
    mm2s.xo \
    s2mm.xo \
    libadf.a \
    -o link.xsa

In [ ]:
!bash src/link.sh

## 8. パッケージ

ハードウェアリンクしたxclbinファイルとAIEグラフをパッケージ化します。

In [ ]:
%%writefile src/package.sh
cd build

source ../src/target.sh

v++ \
    --package \
    --target $TARGET \
    --platform $PLATFORM \
    --package.boot_mode=ospi \
    link.xsa \
    libadf.a \
    -o vadd.xclbin

In [ ]:
!bash src/package.sh

## 9. ホストプログラムの作成

XRT Native APIを使ってホストプログラムをC++で記述します。

In [ ]:
%%writefile src/app.cpp
#include <iostream>

#include <xrt/xrt_bo.h>
#include <xrt/xrt_device.h>
#include <xrt/xrt_kernel.h>

int main(int argc, char** argv)
{
    const int device_index = 0;
    const std::string xclbin_file = argv[1];

    std::cout << "(1) Open device" << std::endl;
    auto device = xrt::device(device_index);
    
    std::cout << "(2) Load xclbin, " << xclbin_file << std::endl;
    auto uuid = device.load_xclbin(xclbin_file);

    std::cout << "(3) Create kernels" << std::endl;
    auto mm2s_1 = xrt::kernel(device, uuid, "mm2s:{mm2s_1}");
    auto mm2s_2 = xrt::kernel(device, uuid, "mm2s:{mm2s_2}");
    auto s2mm_1 = xrt::kernel(device, uuid, "s2mm:{s2mm_1}");

    std::cout << "(4) Create buffer objects" << std::endl;
    auto bo_1 = xrt::bo(device, sizeof(int) * 4, mm2s_1.group_id(0));
    auto bo_2 = xrt::bo(device, sizeof(int) * 4, mm2s_2.group_id(0));
    auto bo_3 = xrt::bo(device, sizeof(int) * 4, s2mm_1.group_id(0));

    std::cout << "(5) Map host-side buffer pointers to user space" << std::endl;
    auto buf_1 = bo_1.map<int*>();
    auto buf_2 = bo_2.map<int*>();
    auto buf_3 = bo_3.map<int*>();

    for (int i = 0; i < 4; i++) {
        buf_1[i] = i + 2;
        buf_2[i] = i * i;
    }

    std::cout << "(6) Sync bo to device" << std::endl;
    bo_1.sync(XCL_BO_SYNC_BO_TO_DEVICE);
    bo_2.sync(XCL_BO_SYNC_BO_TO_DEVICE);

    std::cout << "(7) Run kernels" << std::endl;
    auto mm2s_1_run = mm2s_1(bo_1, nullptr, 4);
    auto mm2s_2_run = mm2s_2(bo_2, nullptr, 4);
    auto s2mm_1_run = s2mm_1(bo_3, nullptr, 4);

    std::cout << "(8) Wait for kernels to finish" << std::endl;
    mm2s_1_run.wait();
    mm2s_2_run.wait();
    s2mm_1_run.wait();

    std::cout << "(9) Sync bo from device" << std::endl;
    bo_3.sync(XCL_BO_SYNC_BO_FROM_DEVICE);
    
    std::cout << "(10) Output result" << std::endl;
    for (int i = 0; i < 4; i++) {
        std::cout << buf_3[i] << std::endl;
    }
}

ホストプログラムをビルドします。

In [ ]:
%%writefile src/build_host.sh
cd build

g++ \
    -o app \
    ../src/app.cpp \
    -I/opt/xilinx/xrt/include \
    -L/opt/xilinx/xrt/lib \
    -lxrt_coreutil \
    -pthread

In [ ]:
!bash src/build_host.sh

## 10. ホストプログラムの実行

ハードウェアエミュレーションがターゲットの場合はホストプログラムを実行する前に準備が必要です。

xclbinファイルの生成は時間がかかるのでACRiルームでは事前にビルドしたファイルを提供しています。

ホストプログラムにxclbinファイルを引数で渡して実行します。

In [ ]:
%%writefile src/run.sh
cd build

# ターゲットがhw_emuのときエミュレーションコンフィギュレーションファイルを作成し、
# XCL_EMULATION_MODE環境変数をhw_emuに設定する
source ../src/target.sh
if [[ $TARGET == "hw_emu" ]]; then
    test ! -e emconfig.json && emconfigutil --platform $PLATFORM
    export XCL_EMULATION_MODE=hw_emu
fi

XCLBIN=vadd.xclbin

# 実行対象のxclbinが存在しなければpre-builtを使用する
if [[ ! -e $XCLBIN ]] ; then
    XCLBIN=../pre-built/vadd.$TARGET.xclbin
fi

# ホストプログラムを実行
time ./app $XCLBIN

In [ ]:
!bash src/run.sh

期待通りに出力されれば成功です。本チュートリアルはここまでとなります。